<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/rule_ml_pair_pron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Импорт**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
from IPython.display import SVG, display, HTML

In [3]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Mounted at ./content


In [4]:
import spacy
import numpy as np

import gensim
from gensim.models import Word2Vec 
from gensim.models import FastText

from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [6]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content  sample_data
.file-revisions-by-id
MyDrive
.shortcut-targets-by-id
.Trash


In [7]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
pip install pymorphy2[fast]

     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 8.2MB 8.2MB/s 
     |████████████████████████████████| 378kB 33.7MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=860450 sha256=389bdee03e4c62d14d8433d1b3655645f14b7c8f01b8e876fe56585655c19c2a
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


## **Обработка данных**

In [9]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-31 10:46:46--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt’

neuroleptic_antidep 100%[===================>]  11.12M  --.-KB/s    in 0.1s    

2021-05-31 10:46:47 (74.6 MB/s) - ‘neuroleptic_antidepr2.txt’ saved [11655409/11655409]

--2021-05-31 10:46:47--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443.

In [10]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv

--2021-05-31 10:46:49--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-31 10:46:49 ERROR 404: Not Found.

--2021-05-31 10:46:49--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100469 (2.0M) [text/plain]
Saving to: ‘annotation_table.csv’

annotation_table.cs 100%[===================>]   2.00M  --.-KB/s    in 0.1s

In [11]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [12]:
posts = my_txt.split('----')

In [13]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [14]:
morph.parse('он')[0].normal_form

'он'

## **Обучение векторизатора**

In [15]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]


sentences_tokenized = [tokenizer.tokenize(morph.parse(i)[0].normal_form) for i in sentences_tokenized]
ft = FastText(sentences_tokenized, window=5, min_count=5, workers=4,sg=1)

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [16]:
count = 0
with open('annotation_table.csv') as csvfile:
  with open('annotations.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    for new_row in csvfile:
      count +=1
      # print(i.split('\t'))
      if len(new_row.split('\t')) > 19:
        new_row = new_row.split('\t')[:20]
        new_row[-1] = new_row[-1][:-1] # cut \n
      writer.writerow(new_row)
count

27019

In [17]:
# # with open('coref_df_anns60k.csv') as csvfile:
# with open('new_anns60k.csv') as csvfile:
#   reader = csv.reader(csvfile, delimiter='\t')
#   with open('new_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     c = 0
#     for row in reader:
#       c +=1
#       new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
#       if len(new_row) > 9:
#         new_row = new_row[:9]
#       else:
#         new_row = [0 for i in range(9)]
#       writer.writerow(new_row)
# c


In [18]:
# !rm new_anns60k.csv.1 new_anns60k.csv.2 new_df_anns60k.csv new_anns60k.csv
!ls
# !rm annotation_table.csv.2 annotation_table.csv.1 annotation_table.csv annotations.csv

annotations.csv       hv-info_gepatit2.txt	 sample_data
annotation_table.csv  neuroleptic_antidepr2.txt
content		      rakpobedim_ru2.txt


In [19]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('annotations.csv', delimiter='\t')
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27013,и,0,22,68.0,0.0,2385.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0,0.0,NaN
27014,таможенные,0,22,68.0,0.0,2386.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0,0.0,plur
27015,пошлины,0,22,68.0,0.0,2387.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0,0.0,plur
27016,как,0,22,68.0,0.0,2388.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0,0.0,NaN


In [20]:
tok_list = []
for token in anns_df.TOKEN:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['TOKEN_VECT'] = tok_list


tok_list = []
for token in anns_df.HEAD:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['HEAD_VECT'] = tok_list

In [21]:
anns_df = anns_df.loc[(anns_df['TOPIC_NUM'].isin(list(range(30)))) | (anns_df['SENT_NUM'].isin(list(range(10000))))]
anns_df = anns_df.reset_index().drop(['index'], axis=1)

In [22]:
topics = list(set(anns_df.TOPIC_NUM.values))
topics
sent_vect_list = []
# номера предложений топика
for topic_num in topics:

  sents_in_topic = list(set(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num].SENT_NUM))
  topic_len = len(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num])
  for sent_num in sents_in_topic:
    tok_vects = anns_df.loc[(anns_df['TOPIC_NUM'] == topic_num) & (anns_df['SENT_NUM'] == sent_num)].TOKEN_VECT
    try:
      sent_vect = sum(list(tok_vects))/len(tok_vects)
    except ZeroDivisionError:
      sent_vect = 0
      print(anns_df.loc[(anns_df['SENT_NUM'] == sent_num)])
    for i in range(len(tok_vects)):
      sent_vect_list.append(sent_vect)
len(sent_vect_list)
# если не класть нулевых предложений - будет длина датафрейма, как он раньше импортировлся, но поздно.

26497

In [23]:
anns_df['SENT_VECT'] = sent_vect_list

Проверим данные

In [24]:
anns_df.SENT_NUM[18]

1.0

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

Добавим размеченные данные, и посмотрим, что получилось.

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [25]:
# df = anns_df.iloc[:15000,:]
# df = anns_df.iloc[:17000,:]
df = anns_df.iloc[:18000,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [26]:
len(df.loc[df['NER'] == 1])

628

Из них местоимений (прономинальные анафоры).

In [27]:
len(df.loc[df['NER'] == 3])

117

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [28]:
len(df.loc[df['NER'] == 2])

246

In [29]:
len(df.loc[(df['NER'].isin([2,3]))])

363

In [30]:
df.loc[df['COREFERENCE_CLUSTER']==0]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,"[0.25268957, 0.5728969, -0.07278406, -0.328561...","[0.25268957, 0.5728969, -0.07278406, -0.328561...","[0.51110846, 0.40224165, 0.15548122, -0.290461..."
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing,"[-0.25193167, 0.42567253, -0.436869, 1.0471027...","[0.25268957, 0.5728969, -0.07278406, -0.328561...","[0.51110846, 0.40224165, 0.15548122, -0.290461..."
5,не,0,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,"[0.5424425, -0.1306316, -1.0794508, 0.22822134...","[0.9260997, 0.71420664, 0.030847095, 0.0325862...","[0.51110846, 0.40224165, 0.15548122, -0.290461..."
6,знаю,0,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,"[0.9260997, 0.71420664, 0.030847095, 0.0325862...","[0.9260997, 0.71420664, 0.030847095, 0.0325862...","[0.51110846, 0.40224165, 0.15548122, -0.290461..."
7,стоило,0,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,"[0.3283123, 0.31369743, 0.39008793, 0.6223606,...","[0.9260997, 0.71420664, 0.030847095, 0.0325862...","[0.51110846, 0.40224165, 0.15548122, -0.290461..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,схемы,0,21,6.0,0.0,214.0,inan,gent,femn,NaN,NOUN,obj,использовать,NaN,NaN,NaN,INFN,0,0.0,sing,"[0.83858615, 0.8730295, 0.29178205, -0.0048502...","[0.46143797, 1.2322521, -0.22042039, 0.2872439...","[0.4104073, 0.19052018, -0.04674826, -0.051933..."
17996,рекомендованные,0,21,6.0,0.0,215.0,NaN,nomn,NaN,NaN,PRTF,acl,схемы,inan,gent,femn,NOUN,0,0.0,plur,"[0.5752893, 0.23874398, 0.4425047, 0.2141485, ...","[0.83858615, 0.8730295, 0.29178205, -0.0048502...","[0.4104073, 0.19052018, -0.04674826, -0.051933..."
17997,в,0,21,6.0,0.0,216.0,NaN,NaN,NaN,NaN,PREP,case,версиях,inan,loct,femn,NOUN,0,0.0,NaN,"[0.14497088, -0.44387242, 0.28283212, 0.105165...","[0.36097062, 0.1224831, 0.4029402, 0.4662715, ...","[0.4104073, 0.19052018, -0.04674826, -0.051933..."
17998,более,0,21,6.0,0.0,217.0,NaN,NaN,NaN,NaN,ADVB,advmod,ранних,NaN,gent,NaN,ADJF,0,0.0,NaN,"[0.036348537, 0.05407755, 0.5619486, 0.3759330...","[0.57462966, 0.32479644, 0.2861206, 0.52790564...","[0.4104073, 0.19052018, -0.04674826, -0.051933..."


In [31]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [32]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing,"[-0.25193167, 0.42567253, -0.436869, 1.0471027...","[0.25268957, 0.5728969, -0.07278406, -0.328561...","[0.51110846, 0.40224165, 0.15548122, -0.290461..."
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.16339429, 0.96756715, 0.5229744, 0.18487968...","[-0.25193167, 0.42567253, -0.436869, 1.0471027...","[0.51110846, 0.40224165, 0.15548122, -0.290461..."
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,"[0.15600604, 0.7841326, 0.5179352, 0.083175935...","[0.16339429, 0.96756715, 0.5229744, 0.18487968...","[0.51110846, 0.40224165, 0.15548122, -0.290461..."
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,"[-0.007313758, 0.97775805, 0.35898367, -0.0040...","[0.15600604, 0.7841326, 0.5179352, 0.083175935...","[0.51110846, 0.40224165, 0.15548122, -0.290461..."
21,лекарством,0,1,1.0,1.0,21.0,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,2,0.0,sing,"[-0.19725947, -0.8301398, 0.42346287, 0.581301...","[0.29286018, -0.05770154, -0.2154081, 0.211047...","[0.51110846, 0.40224165, 0.15548122, -0.290461..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17918,вел,0,21,4.0,2.0,137.0,NaN,NaN,masc,NaN,VERB,ROOT,вел,NaN,NaN,masc,VERB,1,136.0,sing,"[1.9802724, -0.2432985, 1.0763333, 0.058015738...","[1.9802724, -0.2432985, 1.0763333, 0.058015738...","[0.42137554, 0.10035562, -0.04355511, 0.020379..."
17929,соф,0,21,4.0,3.0,148.0,inan,gent,femn,NaN,NOUN,nmod,форума,inan,gent,masc,NOUN,1,148.0,plur,"[1.7520199, -0.2151485, -0.4477236, -0.5238399...","[1.1703207, 0.65470886, 0.081057124, -0.151169...","[0.42137554, 0.10035562, -0.04355511, 0.020379..."
17930,даке,0,21,4.0,3.0,149.0,inan,datv,femn,NaN,NOUN,nmod,соф,inan,gent,femn,NOUN,1,149.0,sing,"[1.2724912, -0.30434263, -0.7535911, 0.1575441...","[1.7520199, -0.2151485, -0.4477236, -0.5238399...","[0.42137554, 0.10035562, -0.04355511, 0.020379..."
17937,соф,0,21,4.0,3.0,156.0,inan,gent,femn,NaN,NOUN,nsubj,вел,NaN,NaN,masc,VERB,1,156.0,plur,"[1.7520199, -0.2151485, -0.4477236, -0.5238399...","[1.9802724, -0.2432985, 1.0763333, 0.058015738...","[0.42137554, 0.10035562, -0.04355511, 0.020379..."


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [33]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.16339429, 0.96756715, 0.5229744, 0.18487968...","[-0.25193167, 0.42567253, -0.436869, 1.0471027...","[0.51110846, 0.40224165, 0.15548122, -0.290461..."
576,зелдокс,0,2,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.6621809, 0.7586567, 0.3330336, 0.08735275, ...","[-0.25193167, 0.42567253, -0.436869, 1.0471027...","[0.28453973, 0.3323144, 0.025812093, -0.011060..."
1870,оланзапин,0,3,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.19266027, 1.0034709, 0.52841496, -0.2384468...","[-0.25193167, 0.42567253, -0.436869, 1.0471027...","[0.33296654, 0.31693396, 0.051829126, -0.07547..."
5555,галоперидол,0,4,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,1,2.0,sing,"[0.16471279, 1.2324785, 0.119701385, -0.259405...","[0.25268957, 0.5728969, -0.07278406, -0.328561...","[0.36267462, 0.28972438, 0.09850212, 0.0950351..."
6200,тиаприд,0,5,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,3.0,sing,"[0.6175694, 0.6850844, 0.6764492, -0.132946, 0...","[-0.25193167, 0.42567253, -0.436869, 1.0471027...","[0.25782827, 0.32827875, 0.16085956, 0.0418848..."
8450,сероквель,0,6,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.13855864, 1.076585, 0.5123146, 0.14218268, ...","[-0.25193167, 0.42567253, -0.436869, 1.0471027...","[0.21778692, 0.3792418, 0.12544088, -0.0268611..."
8541,день,0,7,0.0,0.0,2.0,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,0,0.0,sing,"[0.12172692, 0.3454211, -0.603357, 0.28699544,...","[0.12172692, 0.3454211, -0.603357, 0.28699544,...","[0.21778692, 0.3792418, 0.12544088, -0.0268611..."
11117,500,0,8,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0,0.0,NaN,"[0.46391627, 0.02972782, 1.1044209, 0.4751326,...","[0.22294763, 0.5148267, 0.83663905, -0.1665857...","[0.2776875, 0.3986665, 0.20332725, 0.23640172,..."
11787,капецитабин,0,10,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,1,2.0,sing,"[0.061593752, 1.1328725, 0.09764638, 0.0438966...","[0.061593752, 1.1328725, 0.09764638, 0.0438966...","[0.26492098, 0.28021815, 0.1546248, 0.02828150..."
11806,иресса,0,11,0.0,0.0,2.0,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,1,2.0,NaN,"[0.46399778, 0.6185177, 0.8586254, -0.40073916...","[1.4263701, 0.7667352, -0.22843324, -0.1842069...","[0.26492098, 0.28021815, 0.1546248, 0.02828150..."


In [34]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [35]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
# head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [36]:
df.TOKEN_VECT[710]

0

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [37]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [38]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
# make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [39]:
df.TOKEN[1]

'препарат'

In [40]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на обработанные данные.

In [41]:
df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,0.516436,0.549628,0.934114
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing,0.592381,0.549628,0.934114
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,0.743508,0.857506,0.934114
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,0.770621,0.578861,0.934114
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,0.750560,0.632106,0.934114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,схемы,0,21,6.0,0.0,214.0,inan,gent,femn,NaN,NOUN,obj,использовать,NaN,NaN,NaN,INFN,0,0.0,sing,0.458832,0.560726,0.940940
17996,рекомендованные,0,21,6.0,0.0,215.0,NaN,nomn,NaN,NaN,PRTF,acl,схемы,inan,gent,femn,NOUN,0,0.0,plur,0.627940,0.403358,0.940940
17997,в,0,21,6.0,0.0,216.0,NaN,NaN,NaN,NaN,PREP,case,версиях,inan,loct,femn,NOUN,0,0.0,NaN,0.458147,0.484427,0.940940
17998,более,0,21,6.0,0.0,217.0,NaN,NaN,NaN,NaN,ADVB,advmod,ранних,NaN,gent,NaN,ADJF,0,0.0,NaN,0.496711,0.548412,0.940940


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [42]:
# df.SENT_NUM = [int(i) for i in df['SENT_NUM']]
# df.POST_NUM = [int(i) for i in df['POST_NUM']]
# # df.TOKEN_NUM = [float(i) for i in df['TOKEN_NUM']]
df.TOPIC_NUM = [int(i) for i in df['TOPIC_NUM']]
df.IS_ANSWER = [int(i) for i in df['TOPIC_NUM']]
# df.NER = [int(i) for i in df['NER']]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [43]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [44]:
df.loc[df['TOKEN_NUM'] == 344 ]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
344,рисполепта,1,1,2.0,4.0,344.0,inan,nomn,femn,NaN,NOUN,nmod,приеме,inan,loct,masc,NOUN,1,344.0,sing,0.737272,0.544474,0.928984
918,вышло,2,2,6.0,3.0,344.0,NaN,NaN,neut,NaN,VERB,ROOT,вышло,NaN,NaN,neut,VERB,0,0.0,sing,0.530852,0.522893,0.973480
2212,в,3,3,5.0,8.0,344.0,NaN,NaN,NaN,NaN,PREP,case,использовании,inan,loct,neut,NOUN,0,0.0,NaN,0.458147,0.553305,0.983415
5897,и,4,4,6.0,7.0,344.0,NaN,NaN,NaN,NaN,CONJ,cc,выписывались,NaN,NaN,NaN,VERB,0,0.0,NaN,0.440497,0.502511,0.973785
6542,в,5,5,17.0,0.0,344.0,NaN,NaN,NaN,NaN,PREP,case,мозгу,inan,loc2,masc,NOUN,0,0.0,NaN,0.458147,0.433494,0.971879
8883,их,7,7,0.0,32.0,344.0,NaN,accs,NaN,3per,NPRO,obj,принимать,NaN,NaN,NaN,INFN,0,0.0,plur,0.482987,0.523173,0.988494
11513,отвратно,9,9,0.0,39.0,344.0,NaN,NaN,NaN,NaN,ADVB,advmod,перенесла,NaN,NaN,femn,VERB,0,0.0,NaN,0.522887,0.551091,0.971773
12936,будет,15,15,0.0,31.0,344.0,NaN,NaN,NaN,3per,VERB,advcl,проходить,NaN,NaN,NaN,INFN,0,0.0,sing,0.486936,0.635214,0.954680
15278,ковид,16,16,9.0,0.0,344.0,inan,nomn,masc,NaN,NOUN,conj,наименование,inan,nomn,neut,NOUN,0,0.0,sing,0.603219,0.508716,0.958730
16465,сказал,18,18,7.0,2.0,344.0,NaN,NaN,masc,NaN,VERB,acl:relcl,парень,anim,nomn,masc,NOUN,0,0.0,sing,0.500202,0.500806,0.951854


In [45]:

topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)]


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
5,не,1,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,0.454022,0.498430,0.934114
6,знаю,1,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.532011,0.498430,0.934114
7,стоило,1,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.537537,0.498430,0.934114
8,ли,1,1,1.0,0.0,8.0,NaN,NaN,NaN,NaN,CONJ,advmod,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.516377,0.501349,0.934114
9,мне,1,1,1.0,0.0,9.0,NaN,datv,NaN,1per,NPRO,iobj,стоило,NaN,NaN,neut,VERB,0,0.0,sing,0.512965,0.501349,0.934114
10,писать,1,1,1.0,0.0,10.0,NaN,NaN,NaN,NaN,INFN,csubj,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.527906,0.501349,0.934114


In [46]:
for i in df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)].values:
  print(i[-5])

0.0
0.0
0.0
0.0
0.0
0.0


In [47]:
# анафоры
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)
& (df['NER'] > 1)].values[0]

array(['лекарством', 1, 1, 1.0, 1.0, 21.0, 'inan', 'ablt', 'neut', nan,
       'NOUN', 'conj', 'нет', nan, nan, nan, 'PRED', 2, 0.0, 'sing',
       0.4623730480670929, 0.5905115604400635, 0.9341139793395996],
      dtype=object)

In [48]:
def rule_eval(my_X, my_y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(my_X)):
    if my_X[i] != 0:
      if my_X[i] == my_y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif my_X[i] == 0:
      if my_X[i] == my_y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')


In [49]:
df.loc[(df['POS'] == 5) &  (df['PERSON'] == 1)].iloc[30:40, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT


In [50]:
topics = list(set(df.TOPIC_NUM.values))
topics
# all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [51]:
for_factorize = 'ANIMACY 	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS NUMBER'.split()
for_factorize

mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
# mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [52]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      current_anaphors = sent.loc[(sent['NER'] > 1)]
      # current_anaphors = sent.loc[(sent['POS'] == 5) &  (sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                  anfr[8] == current_antecedents.GENDER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            # в противном случае присваиваем нулевой класс
            post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                          & (df['POST_NUM'] == post_num) \
                          & (df['SENT_NUM'] <= sent_num)]

            current_antecedents = post.loc[(post['NER'] == 1) ]

            # если его нету в посте, выбираем первый же антецедент из предыдущих постов
            # топика(сверху вниз)
            if not list(current_antecedents.TOKEN):
              topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
              current_antecedents = topic.loc[(topic['NER'] == 1) ]
              # проверяем, а антецеденты-то есть?
              if not current_antecedents.TOKEN.values.any():
                  # присваиваем нулевой класс, если нет даже в топике
                  ann_clusters.append(anfr[-5])
                  # и класс, который определило правило(антецедента из предложения)
                  # для оценки
                  my_clusters.append(0.0)
                  continue
              # ну и если они есть
              # это можно убрать в отдельную функцию
              for anfr in current_anaphors.values:
                # проверяем сходимость по числу
                # try:
                if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                        anfr[8] == current_antecedents.GENDER.values[0]:
                  # добавляем в список размеченный класс
                  ann_clusters.append(anfr[-5])
                  # и класс который определило правило(антецедента из предложения)
                  my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
                  # print(ann_clusters, my_clusters, 'am I here?')
                else:
                  # в противном случае присваиваем нулевой класс
                  ann_clusters.append(anfr[-5])
                  # и класс, который определило правило(антецедента из предложения)
                  my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 382.0, 2.0, 2.0, 440.0, 442.0, 2.0, 2.0, 540.0, 2.0, 3.0, 11.0, 11.0, 11.0, 46.0, 108.0, 108.0, 182.0, 182.0, 182.0, 182.0, 0.0, 0.0, 0.0, 649.0, 650.0, 658.0, 658.0, 658.0, 693.0, 755.0, 755.0, 829.0, 829.0, 829.0, 829.0, 0.0, 0.0, 0.0, 0.0, 0.0, 26.0, 0.0, 0.0, 0.0, 0.0, 0.0, 356.0, 0.0, 0.0, 0.0, 533.0, 551.0, 575.0, 593.0, 0.0, 0.0, 0.0, 830.0, 0.0, 0.0, 0.0, 0.0, 1160.0, 1355.0, 1379.0, 1397.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1844.0, 1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2155.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2711.0, 2711.0, 2711.0, 2766.0, 2766.0, 2766.0, 2766.0, 0.0, 0.0, 0.0, 0.0, 3059.0, 3125.0, 3125.0, 0.0, 3270.0, 3270.0, 3350.0, 3350.0, 3350.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 5.0, 5.0, 2.0, 114.0, 114.0, 0.0, 174.0, 174.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 287.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 449.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 3.0, 10.0, 10.0, 14.0, 14.0, 27.0, 27.0, 41.0, 41.0, 41.0, 2.0, 2.0, 135.0, 135.

In [53]:
rule_eval(my_clusters, ann_clusters)

114  true positive
97  false positive
103  true negative
109  false negative
---------
0.5402843601895735  presicion
0.5112107623318386  recall
0.5253456221198156  f-score


In [54]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      current_anaphors = sent.loc[(sent['NER'] > 1)]
      # current_anaphors = sent.loc[(sent['POS'] == 5) &  (sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
            current_antecedents = topic.loc[(topic['NER'] == 1) ]
            # проверяем, а антецеденты-то есть?
            if not current_antecedents.TOKEN.values.any():
                # присваиваем нулевой класс, если нет даже в топике
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                # для оценки
                my_clusters.append(0.0)
                continue
            # ну и если они есть
            # это можно убрать в отдельную функцию
            for anfr in current_anaphors.values:
              # проверяем сходимость по числу
              # try:
              if anfr[-4] == current_antecedents.NUMBER.values[0]:
                # добавляем в список размеченный класс
                ann_clusters.append(anfr[-5])
                # и класс который определило правило(антецедента из предложения)
                my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
                # print(ann_clusters, my_clusters, 'am I here?')
              else:
                # в противном случае присваиваем нулевой класс
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[2.0, 4.0, 2.0, 0.0, 0.0, 344.0, 382.0, 2.0, 2.0, 440.0, 442.0, 2.0, 2.0, 540.0, 2.0, 3.0, 11.0, 11.0, 11.0, 46.0, 108.0, 108.0, 182.0, 182.0, 182.0, 182.0, 375.0, 434.0, 461.0, 649.0, 650.0, 658.0, 658.0, 658.0, 693.0, 755.0, 755.0, 829.0, 829.0, 829.0, 829.0, 1022.0, 1081.0, 1081.0, 1142.0, 1142.0, 0.0, 3653.0, 26.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3653.0, 3653.0, 3653.0, 183.0, 0.0, 0.0, 0.0, 2.0, 2.0, 533.0, 551.0, 575.0, 593.0, 679.0, 686.0, 0.0, 3653.0, 0.0, 3653.0, 830.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3653.0, 3653.0, 3653.0, 987.0, 0.0, 1344.0, 1355.0, 1379.0, 1397.0, 2.0, 0.0, 1483.0, 1490.0, 0.0, 3653.0, 0.0, 0.0, 0.0, 1750.0, 1750.0, 2.0, 0.0, 1844.0, 0.0, 2.0, 3653.0, 0.0, 2031.0, 2031.0, 2053.0, 2053.0, 0.0, 3653.0, 2058.0, 2084.0, 2155.0, 0.0, 2.0, 3653.0, 0.0, 2156.0, 0.0, 2344.0, 2.0, 0.0, 2711.0, 2711.0, 2711.0, 2766.0, 2766.0, 2766.0, 2766.0, 0.0, 2.0, 0.0, 2.0, 0.0, 0.0, 3125.0, 3125.0, 0.0, 0.0, 3270.0, 3270.0, 3350.0, 3350.0, 3350.0, 0.0,

In [55]:
rule_eval(my_clusters, ann_clusters)

158  true positive
170  false positive
95  true negative
97  false negative
---------
0.4817073170731707  presicion
0.6196078431372549  recall
0.5420240137221269  f-score


In [56]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      # current_anaphors = sent.loc[(sent['NER'] > 1)]
      current_anaphors = sent.loc[(sent['POS'] == 5)& (df['PERSON']== 2)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
            current_antecedents = topic.loc[(topic['NER'] == 1) ]
            # проверяем, а антецеденты-то есть?
            if not current_antecedents.TOKEN.values.any():
                # присваиваем нулевой класс, если нет даже в топике
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                # для оценки
                my_clusters.append(0.0)
                continue
            # ну и если они есть
            # это можно убрать в отдельную функцию
            for anfr in current_anaphors.values:
              # проверяем сходимость по числу
              # try:
              if anfr[-4] == current_antecedents.NUMBER.values[0]:
                # добавляем в список размеченный класс
                ann_clusters.append(anfr[-5])
                # и класс который определило правило(антецедента из предложения)
                my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
                # print(ann_clusters, my_clusters, 'am I here?')
              else:
                # в противном случае присваиваем нулевой класс
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 0.0, 2.0, 0.0, 2.0, 2.0, 0.0, 3585.0, 3585.0, 3595.0, 3595.0, 2.0, 527.0, 527.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 272.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2215.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [57]:
rule_eval(my_clusters, ann_clusters)

0  true positive
37  false positive
73  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [58]:
 df.loc[(df['PERSON']== 2) & (df['POS']== 5)]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
242,вас,1,1,1.0,25.0,242.0,-1,3,-1,2,5,1,111,-1,-1,-1,0,0,0.0,0,0.514456,0.497663,0.950532
303,вами,1,1,2.0,1.0,303.0,-1,5,-1,2,5,10,130,-1,1,2,11,0,0.0,0,0.446992,0.551102,0.911115
338,вы,1,1,2.0,4.0,338.0,-1,0,-1,2,5,12,146,-1,-1,-1,0,0,0.0,0,0.532301,0.457492,0.928984
384,вас,1,1,3.0,1.0,384.0,-1,3,-1,2,5,10,162,-1,-1,-1,0,0,0.0,0,0.514456,0.552244,0.885181
2337,вам,3,3,6.0,2.0,469.0,-1,2,-1,2,5,10,594,-1,-1,-1,0,0,0.0,0,0.497412,0.532586,0.983415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16396,вы,18,18,6.0,6.0,275.0,-1,0,-1,2,5,12,953,-1,-1,-1,0,0,0.0,0,0.532301,0.506261,0.942081
16545,вы,18,18,7.0,6.0,424.0,-1,0,-1,2,5,12,953,-1,-1,-1,0,0,0.0,0,0.532301,0.506261,0.929487
17106,вы,20,20,2.0,0.0,517.0,-1,0,-1,2,5,12,2886,-1,-1,-1,0,0,0.0,0,0.532301,0.472446,0.801062
17670,вас,20,20,4.0,32.0,1081.0,-1,3,-1,2,5,10,2900,-1,0,1,8,0,0.0,0,0.514456,0.437699,0.964452


род

In [59]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      current_anaphors = sent.loc[(sent['NER'] > 1)]
      # current_anaphors = sent.loc[(sent['POS'] == 5)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                  anfr[8] == current_antecedents.GENDER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
            current_antecedents = topic.loc[(topic['NER'] == 1) ]
            # проверяем, а антецеденты-то есть?
            if not current_antecedents.TOKEN.values.any():
                # присваиваем нулевой класс, если нет даже в топике
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                # для оценки
                my_clusters.append(0.0)
                continue
            # ну и если они есть
            # это можно убрать в отдельную функцию
            for anfr in current_anaphors.values:
              # проверяем сходимость по числу
              # try:
              if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                      anfr[8] == current_antecedents.GENDER.values[0]:
                # добавляем в список размеченный класс
                ann_clusters.append(anfr[-5])
                # и класс который определило правило(антецедента из предложения)
                my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
                # print(ann_clusters, my_clusters, 'am I here?')
              else:
                # в противном случае присваиваем нулевой класс
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 382.0, 2.0, 2.0, 440.0, 442.0, 2.0, 2.0, 540.0, 2.0, 3.0, 11.0, 11.0, 11.0, 46.0, 108.0, 108.0, 182.0, 182.0, 182.0, 182.0, 0.0, 0.0, 0.0, 649.0, 650.0, 658.0, 658.0, 658.0, 693.0, 755.0, 755.0, 829.0, 829.0, 829.0, 829.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 26.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 356.0, 0.0, 0.0, 0.0, 533.0, 551.0, 575.0, 593.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 830.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1160.0, 0.0, 0.0, 1379.0, 1397.0, 2.0, 3653.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2155.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2711.0, 2711.0, 2711.0, 2766.0, 2766.0, 2766.0, 2766.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3059.0, 3125.0, 3125.0, 0.0, 0.0, 3270.0, 3270.0, 3350.0, 3350.0, 3350.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 5.0, 5.0, 2.0, 114.0, 114.0, 0.0, 174.0, 174.0, 0.0, 0.0, 0.0, 0.0, 0.

In [60]:
rule_eval(my_clusters, ann_clusters)

114  true positive
100  false positive
133  true negative
178  false negative
---------
0.5327102803738317  presicion
0.3904109589041096  recall
0.4505928853754941  f-score


In [61]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      # current_anaphors = sent.loc[(sent['NER'] > 1)]
      current_anaphors = sent.loc[(sent['POS'] == 5) & (sent['PERSON'] == 2)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                  anfr[8] == current_antecedents.GENDER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
            current_antecedents = topic.loc[(topic['NER'] == 1) ]
            # проверяем, а антецеденты-то есть?
            if not current_antecedents.TOKEN.values.any():
                # присваиваем нулевой класс, если нет даже в топике
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                # для оценки
                my_clusters.append(0.0)
                continue
            # ну и если они есть
            # это можно убрать в отдельную функцию
            for anfr in current_anaphors.values:
              # проверяем сходимость по числу
              # try:
              if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                      anfr[8] == current_antecedents.GENDER.values[0]:
                # добавляем в список размеченный класс
                ann_clusters.append(anfr[-5])
                # и класс который определило правило(антецедента из предложения)
                my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
                # print(ann_clusters, my_clusters, 'am I here?')
              else:
                # в противном случае присваиваем нулевой класс
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [62]:
rule_eval(my_clusters, ann_clusters)

0  true positive
0  false positive
126  true negative
0  false negative
---------
0  presicion
0  recall
0  f-score


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [63]:
for_factorize = 'ANIMACY 	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS NUMBER'.split()
for_factorize


['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS',
 'NUMBER']

In [64]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Int64Index([-1, 0, 1], dtype='int64')


array([0, 1, 2, ..., 0, 0, 0])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [65]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Int64Index([-1, 0, 1], dtype='int64'),
 'CASE': Int64Index([-1, 0, 1, 2, 3, 4, 5, 6, 7, 8], dtype='int64'),
 'DEPENDENCY': Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
             17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
             34, 35],
            dtype='int64'),
 'GENDER': Int64Index([-1, 0, 1, 2], dtype='int64'),
 'HEAD': Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
             ...
             2972, 2973, 2974, 2975, 2976, 2977, 2978, 2979, 2980, 2981],
            dtype='int64', length=2982),
 'HEAD_ANIMACY': Int64Index([-1, 0, 1, 2], dtype='int64'),
 'HEAD_CASE': Int64Index([-1, 0, 1, 2, 3, 4, 5, 6, 7], dtype='int64'),
 'HEAD_GENDER': Int64Index([-1, 0, 1, 2], dtype='int64'),
 'HEAD_POS': Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -1, 14, 15, 16], dtype='int64'),
 'NUMBER': Int64Index([0, 1, -1, 2], dtype='int64'),
 'PERSON': Int64Index([0, -1, 1, 2], dtype=

In [66]:
df.loc[ (df['NER'] == 3.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
475,его,1,1,7.0,1.0,475.0,0,4,1,2,5,11,199,1,3,3,1,3,2.0,1,0.554114,0.348759,0.980417
566,он,1,1,9.0,3.0,566.0,0,1,1,2,5,12,231,0,0,1,0,3,536.0,1,0.538690,0.420141,0.980417
587,он,2,2,1.0,0.0,13.0,0,1,1,2,5,12,235,0,0,1,0,3,11.0,1,0.538690,0.605595,0.980417
594,он,2,2,1.0,1.0,20.0,0,1,1,2,5,12,242,0,0,0,0,3,11.0,1,0.538690,0.601311,0.980417
736,него,2,2,3.0,5.0,162.0,0,2,1,2,5,10,293,0,0,0,0,3,108.0,1,0.592547,0.598253,0.975141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14906,него,15,15,19.0,3.0,2314.0,0,2,1,2,5,21,2453,0,0,0,16,3,2295.0,1,0.592547,0.538139,0.980223
14967,он,16,16,1.0,4.0,33.0,0,1,1,2,5,12,2504,0,0,1,0,3,0.0,1,0.538690,0.667316,0.980223
16328,они,18,18,6.0,2.0,207.0,0,1,0,2,5,12,2507,0,0,0,9,3,0.0,0,0.479294,0.551318,0.948805
16391,его,18,18,6.0,5.0,270.0,0,4,1,2,5,1,1588,0,0,0,5,3,0.0,1,0.554114,0.551750,0.942081


In [67]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,1,1,0.0,0.0,1.0,1,1,1,1,1,1,0,0,0,0,0,2,0.0,1,0.592381,0.549628,0.934114
2,рисперидон,1,1,0.0,0.0,2.0,2,1,1,1,1,2,1,1,1,1,1,1,2.0,1,0.743508,0.857506,0.934114
3,рисполепт,1,1,0.0,0.0,3.0,1,2,2,1,1,3,2,2,1,1,1,1,3.0,0,0.770621,0.578861,0.934114
4,сперидан,1,1,0.0,0.0,4.0,0,0,1,1,2,4,3,1,2,2,1,1,4.0,1,0.750560,0.632106,0.934114
21,лекарством,1,1,1.0,1.0,21.0,1,6,3,1,1,4,7,0,0,0,3,2,0.0,1,0.462373,0.590512,0.934114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17918,вел,21,21,4.0,2.0,137.0,0,0,1,1,0,0,2961,0,0,1,0,1,136.0,1,0.464532,0.422702,0.936932
17929,соф,21,21,4.0,3.0,148.0,1,2,2,1,1,21,2964,1,2,1,1,1,148.0,0,0.483560,0.513021,0.936932
17930,даке,21,21,4.0,3.0,149.0,1,3,2,1,1,21,2945,1,2,2,1,1,149.0,1,0.528332,0.511828,0.936932
17937,соф,21,21,4.0,3.0,156.0,1,2,2,1,1,12,2961,0,0,1,0,1,156.0,0,0.483560,0.422702,0.936932


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [68]:

# train, test = train_test_split(df.loc[(df['NER'] > 1 )], test_size=0.2)
train, test = train_test_split(df.loc[(df['POS'] == 1) &  (df['NER'] == 2)]\
            .append(df.loc[(df['POS'] == 5) & (df['PERSON'] == 2)]))

In [69]:
train = train.reset_index()

In [70]:
test = test.reset_index()

In [71]:
test.head(5)

,index,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,3717,препараты,3,3,22.0,3.0,1849.0,1,1,1,1,1,4,448,2,5,1,1,2,0.0,0,0.516915,0.577193,0.983490
1,13715,химиотерапии,15,15,9.0,1.0,1123.0,1,2,2,1,1,21,1996,1,2,2,1,2,0.0,1,0.619231,0.407544,0.791583
2,14833,ней,15,15,18.0,5.0,2241.0,0,5,2,2,5,10,2485,0,0,2,0,0,0.0,1,0.509351,0.481789,0.980223
3,6732,нейролептик,5,5,21.0,0.0,534.0,1,1,1,1,1,4,237,0,1,1,2,2,521.0,1,0.720855,0.503069,0.968641
4,2704,препарат,3,3,12.0,1.0,836.0,1,1,1,1,1,12,425,0,0,0,9,2,830.0,1,0.592381,0.534198,0.983415


Делаем наборы для обучения моделей.

In [72]:
X, y, X_test, y_test = train.drop(['HEAD_VECT', 'NUMBER', 'TOKEN',  'GENDER', 'POS', 'IS_ANSWER', 'PERSON', \
                                   'TOPIC_NUM', 'CASE',   'DEPENDENCY',   'HEAD_ANIMACY',	'HEAD_CASE',\
                                   	'HEAD_GENDER',  'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop(['HEAD_VECT', 'NUMBER','TOKEN','GENDER', 'POS', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', \
                                      'CASE',  'DEPENDENCY',     'HEAD_ANIMACY',	'HEAD_CASE',\
                                      'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18],\
                                       test.COREFERENCE_CLUSTER

In [73]:
len(df.loc[(df['NER'] > 1 )]), len(X_test), len(X), len(df.loc[(df['NER'] > 1 ) & (df['COREFERENCE_CLUSTER'] == 0 )])

(363, 106, 316, 141)

In [74]:
X.NER = [float(i) for i in X.NER]
X_test.NER = [float(i) for i in X_test.NER]
X.drop(['COREFERENCE_CLUSTER'], axis=1)

,index,POST_NUM,SENT_NUM,TOKEN_NUM,NER,TOKEN_VECT,SENT_VECT
0,10613,15.0,3.0,2074.0,0.0,0.481586,0.983066
1,17883,3.0,0.0,102.0,0.0,0.545082,0.948371
2,587,1.0,0.0,13.0,3.0,0.538690,0.980417
3,1300,12.0,6.0,726.0,2.0,0.592381,0.939295
4,10190,8.0,19.0,1651.0,0.0,0.425507,0.981186
...,...,...,...,...,...,...,...
311,6370,8.0,2.0,172.0,3.0,0.554114,0.972595
312,14440,14.0,2.0,1848.0,3.0,0.554114,0.884193
313,5139,37.0,4.0,3271.0,3.0,0.538690,0.983140
314,8099,48.0,5.0,1901.0,2.0,0.470694,0.988962


In [75]:
y

0         0.0
1         0.0
2        11.0
3         0.0
4         0.0
        ...  
311     135.0
312    1827.0
313    3270.0
314    1884.0
315    2156.0
Name: COREFERENCE_CLUSTER, Length: 316, dtype: float64

Обучаем модели.

In [76]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.6886792452830188

In [77]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.6698113207547169

In [78]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.6698113207547169

In [79]:

regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)


1.0

In [80]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.4528301886792453

In [81]:

knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.7075471698113207

In [82]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [83]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [84]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [85]:
# my_eval(list(naive_pred), list(y_test))

In [86]:
my_eval(list(pred_knn), list(y_test))

28  true positive
26  false positive
47  true negative
5  false negative
---------
0.5185185185185185  presicion
0.8484848484848485  recall
0.6436781609195402  f-score


In [87]:
my_eval(list(pred_entropy), list(y_test))

23  true positive
35  false positive
48  true negative
0  false negative
---------
0.39655172413793105  presicion
1.0  recall
0.5679012345679012  f-score


In [88]:
my_eval(list(pred), list(y_test))

25  true positive
33  false positive
48  true negative
0  false negative
---------
0.43103448275862066  presicion
1.0  recall
0.6024096385542169  f-score


In [89]:
my_eval(list(pred_randf), list(y_test))

23  true positive
32  false positive
48  true negative
3  false negative
---------
0.41818181818181815  presicion
0.8846153846153846  recall
0.5679012345679012  f-score


In [90]:
my_eval(list(pred_regr), list(y_test))

20  true positive
86  false positive
0  true negative
0  false negative
---------
0.18867924528301888  presicion
1.0  recall
0.3174603174603175  f-score


In [91]:
my_eval(list(pred_svm), list(y_test))

0  true positive
0  false positive
48  true negative
58  false negative
---------
0  presicion
0.0  recall
0  f-score


In [92]:
# df['TOPIC_NUM'] == 1
# df.loc[df['TOPIC_NUM'] == 1]

In [93]:
df.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', \
         'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'HEAD_ANIMACY',	'HEAD_CASE',	\
         'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18]

,TOKEN_NUM,NER,COREFERENCE_CLUSTER,TOKEN_VECT,SENT_VECT
0,0.0,0,0.0,0.516436,0.934114
1,1.0,2,0.0,0.592381,0.934114
2,2.0,1,2.0,0.743508,0.934114
3,3.0,1,3.0,0.770621,0.934114
4,4.0,1,4.0,0.750560,0.934114
...,...,...,...,...,...
17995,214.0,0,0.0,0.458832,0.940940
17996,215.0,0,0.0,0.627940,0.940940
17997,216.0,0,0.0,0.458147,0.940940
17998,217.0,0,0.0,0.496711,0.940940


In [94]:
len(df.loc[(df['POS'] == 5) & (df['NER'] > 1)].TOKEN)

120

In [95]:
df.loc[(df['COREFERENCE_CLUSTER'] != 0) & (df['NER'] > 1)]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
366,препарата,1,1,2.0,5.0,366.0,1,2,1,1,1,21,154,1,2,2,1,2,344.0,1,0.511768,0.458757,0.928984
393,препарата,1,1,3.0,1.0,393.0,1,2,1,1,1,21,164,1,4,3,1,2,382.0,1,0.511768,0.479956,0.885181
421,препарат,1,1,5.0,0.0,421.0,1,1,1,1,1,0,1,1,1,1,1,2,2.0,1,0.592381,0.857506,0.980417
429,препарат,1,1,6.0,0.0,429.0,1,1,1,1,1,12,179,0,0,0,0,2,2.0,1,0.592381,0.524470,0.980417
446,последнему,1,1,6.0,2.0,446.0,0,3,1,1,10,7,184,0,0,0,0,2,442.0,1,0.607921,0.511480,0.980417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14440,его,15,15,14.0,2.0,1848.0,0,4,1,2,5,1,1175,0,0,0,0,3,1827.0,1,0.554114,0.560899,0.884193
14478,него,15,15,14.0,6.0,1886.0,0,2,1,2,5,21,2453,0,0,0,16,3,1867.0,1,0.592547,0.538139,0.780539
14733,него,15,15,17.0,8.0,2141.0,0,2,1,2,5,10,2278,0,0,2,0,3,2138.0,1,0.592547,0.582888,0.967740
14906,него,15,15,19.0,3.0,2314.0,0,2,1,2,5,21,2453,0,0,0,16,3,2295.0,1,0.592547,0.538139,0.980223


In [96]:
int(df.loc[(df['TOPIC_NUM'] == 1) & (df['POS'] == 5)].values[0][4])
df.loc[(df['TOPIC_NUM'] == 1) & (df['POS'] == 5)].values[0][5]

9.0

# **ПОПАРНОЕ ИЗВЛЕЧЕНИЕ**

In [97]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')
  # df.loc[(df['POS'] == 1) &  (df['NER'] == 2) & (df['TOPIC_NUM'] == topic_num)]\
  #           .append(df.loc[(df['POS'] == 5) & (df['PERSON'] == 2) & (df['TOPIC_NUM'] == topic_num)]).\
  #           loc[:,['TOKEN_VECT', 'CASE', 'HEAD_VECT', 'NER']]

In [98]:
topics = list(set(df.TOPIC_NUM.values))
topics


pair_anaphors = []
pair_antecedents = []
relation_list = []
counter = 0

for topic_num in topics:

  all_anaphors = df.loc[(df['POS'] == 1) &  (df['NER'] == 2) & (df['TOPIC_NUM'] == topic_num)]\
            .append(df.loc[(df['POS'] == 5) & (df['PERSON'] == 2) & (df['TOPIC_NUM'] == topic_num)])

  all_antecedents = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] == 1)]

  for anaphor in all_anaphors.values:
    # чтобы примеры влезли в память, и модель не переобучилась на антипримерах,
    # возьмем один положительный и два отрицательных класса для каждого анафора
    counter = 0
    for antecedent in all_antecedents.values:
      is_relation = 0

      # print(anaphor[-5], antecedent[5])
      if anaphor[-5] == antecedent[5]:
        is_relation = 1
        # такие срезы, чтобы убрать лишние данные, и в том числе разметку с ответами
        pair_anaphors.append(list(anaphor)[2:6] + list(anaphor)[17:18]\
                             + list(anaphor)[19:23])
        
        pair_antecedents.append(list(antecedent)[2:6] + list(antecedent)[17:18]\
                             + list(antecedent)[19:23])
        
        relation_list.append(is_relation)
      else:
        if counter < 3:

          pair_anaphors.append(list(anaphor)[2:6] + list(anaphor)[17:18]\
                              + list(anaphor)[19:23])
          
          pair_antecedents.append(list(antecedent)[2:6] + list(antecedent)[17:18]\
                              + list(antecedent)[19:23])
          
          relation_list.append(is_relation)
          counter += 1

  # counter += 1

print(pair_anaphors[0][0], pair_antecedents[0][0], relation_list[0])
len(pair_anaphors), len(pair_antecedents), len(relation_list)

1 1 0


(1403, 1403, 1403)

In [99]:
df.loc[(df['TOPIC_NUM'] == 1) & \
              (df['NER'] == 1) & \
              (df['SENT_NUM'] <= 4.0) & \
              (df['POST_NUM'].isin([0.0, 2.0, 3.0]))]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,1,1,0.0,0.0,2.0,2,1,1,1,1,2,1,1,1,1,1,1,2.0,1,0.743508,0.857506,0.934114
3,рисполепт,1,1,0.0,0.0,3.0,1,2,2,1,1,3,2,2,1,1,1,1,3.0,0,0.770621,0.578861,0.934114
4,сперидан,1,1,0.0,0.0,4.0,0,0,1,1,2,4,3,1,2,2,1,1,4.0,1,0.750560,0.632106,0.934114
344,рисполепта,1,1,2.0,4.0,344.0,1,1,2,1,1,21,147,1,6,1,1,1,344.0,1,0.737272,0.544474,0.928984
382,рисперидон,1,1,3.0,0.0,382.0,2,1,1,1,1,1,67,0,0,0,0,1,382.0,1,0.743508,0.591329,0.885181
394,флюанксол,1,1,3.0,2.0,394.0,1,1,1,1,1,12,166,0,0,1,0,1,394.0,1,0.693902,0.473076,0.885181


In [100]:
# можно посмотреть, что получилось
relation_list[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]

In [101]:
pair_anaphors[0]

[1,
 0.0,
 0.0,
 1.0,
 2,
 1,
 0.5923811793327332,
 0.5496277809143066,
 0.9341139793395996]

In [102]:
# проверим, третьим элементом должен быть номер токена
len([i for i in relation_list if i==1])

188

In [103]:
len([i for i in relation_list if i==0])

1215

In [104]:
# все сходится. мы взяли колонки с номерами поста, предложения и токена,\
# колонку NER, число, и вектора
df.loc[df.TOKEN_NUM == 21.0].iloc[[0]]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
21,лекарством,1,1,1.0,1.0,21.0,1,6,3,1,1,4,7,0,0,0,3,2,0.0,1,0.462373,0.590512,0.934114


In [105]:
cosine = [cosine_similarity([pair_anaphors[i]], [pair_antecedents[i]])[0][0] for i in range(len(pair_anaphors))]
# pair_df = pd.DataFrame(columns=['anaphs', 'ants', 'rel'])
pair_df = pd.DataFrame(columns=['cosine', 'rel'])

# pair_df['anaphs'] = pair_anaphors
# pair_df['ants'] = pair_antecedents
pair_df['cosine'] = cosine
pair_df['rel'] = relation_list

print(pair_df.loc[pair_df['cosine'] > 1])
print(pair_df.loc[pair_df['rel'] > 1])
pair_df 

Empty DataFrame
Columns: [cosine, rel]
Index: []
Empty DataFrame
Columns: [cosine, rel]
Index: []


,cosine,rel
0,0.880835,0
1,0.732001,0
2,0.725138,0
3,0.742925,0
4,0.889840,0
...,...,...
1398,0.764756,0
1399,0.812734,0
1400,0.995475,0
1401,0.999458,0


In [106]:
pair_df.loc[pair_df.rel != 0]

,cosine,rel
15,0.999994,1
19,0.999993,1
20,0.665512,1
24,0.665186,1
28,0.664911,1
...,...,...
1284,0.999983,1
1303,0.999998,1
1307,0.999999,1
1320,0.999999,1


In [107]:
# train, test = train_test_split(med_df, test_size=0.3, shuffle=True)
# train = train.reset_index()
# test = test.reset_index()

train, test = train_test_split(pair_df, test_size=0.2, shuffle=True)
train = train.reset_index()
test = test.reset_index()


In [108]:
train

,index,cosine,rel
0,1210,0.990544,0
1,767,0.504160,0
2,1134,0.509550,0
3,896,0.748636,0
4,543,0.999033,0
...,...,...,...
1117,724,0.949579,0
1118,1074,0.990746,0
1119,1367,0.866461,0
1120,116,0.970383,0


In [109]:
test

,index,cosine,rel
0,24,0.665186,1
1,508,0.417005,0
2,869,0.490720,0
3,888,0.993511,0
4,1051,0.929217,0
...,...,...,...
276,500,0.998776,0
277,348,0.752234,0
278,9,0.666103,0
279,353,0.999996,1


In [110]:
train.iloc[:1, 1:2]

,cosine
0,0.990544


In [111]:
test.iloc[:1, 1:2]

,cosine
0,0.665186


In [112]:
X, y, X_test, y_test = train.iloc[:, 1:2], \
                    train.rel, test.iloc[:, 1:2], test.rel

In [113]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')

In [114]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
0  true positive
281  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [115]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
32  true positive
6  false positive
234  true negative
9  false negative
---------
0.8421052631578947  presicion
0.7804878048780488  recall
0.810126582278481  f-score


In [116]:
dtclf = tree.DecisionTreeClassifier(max_depth=2000, )
dtclf.fit(X, y)


print('-----')
pred = dtclf.predict(X_test)
pair_eval(pred, y_test)


-----
32  true positive
10  false positive
230  true negative
9  false negative
---------
0.7619047619047619  presicion
0.7804878048780488  recall
0.7710843373493976  f-score


In [117]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
32  true positive
10  false positive
230  true negative
9  false negative
---------
0.7619047619047619  presicion
0.7804878048780488  recall
0.7710843373493976  f-score


In [118]:
clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
rule_eval(pred, y_test)

-----
34  true positive
10  false positive
230  true negative
7  false negative
---------
0.7727272727272727  presicion
0.8292682926829268  recall
0.7999999999999999  f-score


In [119]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [120]:
# clf = tree.DecisionTreeClassifier(max_depth=2000, )
# clf.fit(X, y)
# clf.score(X_test, y_test)

# **Извлечение номинативных анафоров**

In [121]:
df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
16,проблем,1,1,1.0,1.0,16.0,1,2,2,1,1,12,7,0,0,0,3,0,0.0,0,0.513477,0.590512,0.934114
19,частности,1,1,1.0,1.0,19.0,1,5,2,1,1,15,10,0,0,0,4,0,0.0,1,0.538471,0.461803,0.934114
24,мере,1,1,1.0,2.0,24.0,1,3,2,1,1,15,12,0,0,0,4,0,0.0,1,0.568933,0.534924,0.934114
25,врач,1,1,1.0,2.0,25.0,2,1,1,1,1,12,11,0,0,0,0,0,0.0,1,0.617470,0.526266,0.934114
35,детства,1,1,1.0,3.0,35.0,1,2,3,1,1,10,15,0,0,0,0,0,0.0,1,0.509565,0.317054,0.934114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,месяц,2,2,5.0,1.0,275.0,1,4,1,1,1,10,324,0,0,0,5,0,0.0,1,0.579347,0.525727,0.975141
853,манию,2,2,5.0,1.0,279.0,1,4,2,1,1,10,326,0,0,2,0,0,0.0,1,0.522891,0.557073,0.975141
856,мг,2,2,5.0,1.0,282.0,1,1,1,1,1,10,326,0,0,2,0,0,0.0,1,0.577470,0.557073,0.975141
859,началом,2,2,5.0,2.0,285.0,1,6,3,1,1,10,328,0,0,2,0,0,0.0,1,0.528841,0.465777,0.975141


In [122]:
df.loc[(df['POS'] == 1) & (df['NER'] > 1)] 

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,1,1,0.0,0.0,1.0,1,1,1,1,1,1,0,0,0,0,0,2,0.0,1,0.592381,0.549628,0.934114
21,лекарством,1,1,1.0,1.0,21.0,1,6,3,1,1,4,7,0,0,0,3,2,0.0,1,0.462373,0.590512,0.934114
217,таблетки,1,1,1.0,21.0,217.0,1,4,2,1,1,10,101,0,0,2,0,2,0.0,0,0.529794,0.721434,0.950532
295,таблетки,1,1,2.0,0.0,295.0,1,4,2,1,1,10,101,0,0,2,0,2,0.0,0,0.529794,0.721434,0.817821
366,препарата,1,1,2.0,5.0,366.0,1,2,1,1,1,21,154,1,2,2,1,2,344.0,1,0.511768,0.458757,0.928984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15651,вакцина,16,16,16.0,0.0,717.0,1,1,2,1,1,21,2642,1,1,3,1,2,0.0,1,0.605058,0.479747,0.964830
15692,химиотерапии,16,16,17.0,3.0,758.0,1,2,2,1,1,21,142,2,1,1,1,2,0.0,1,0.619231,0.578048,0.964830
15751,химиотерапий,16,16,18.0,1.0,817.0,1,2,2,1,1,21,2609,1,2,2,1,2,0.0,0,0.672480,0.703884,0.964830
15873,вакцина,16,16,19.0,0.0,939.0,1,1,2,1,1,21,2642,1,1,3,1,2,0.0,1,0.605058,0.479747,0.963957


In [123]:
df.loc[(df['POS'] == 1) & (df['NER'] > 1)].TOKEN.values

array(['препарат', 'лекарством', 'таблетки', 'таблетки', 'препарата',
       'препарата', 'препарат', 'препарат', 'препарат', 'препарат',
       'препарат', 'препарата', 'препарат', 'препарат', 'препарата',
       'препарат', 'препарат', 'препарата', 'препарата', 'препарат',
       'препарат', 'препарата', 'сибазона', 'препарат', 'препарат',
       'препарата', 'лекарства', 'препаратов', 'правозащитники',
       'препарата', 'препарата', 'препарат', 'препарат', 'препарата',
       'лекарства', 'препаратов', 'препарата', 'препарата', 'таблетки',
       'препарата', 'препарат', 'препараты', 'препарата', 'препараты',
       'антипсихотиком', 'препаратов', 'препараты', 'антидепресантом',
       'препаратов', 'препарат', 'препараты', 'лекарств', 'препарата',
       'препарат', 'препарата', 'препарат', 'нейролептика', 'нормотимики',
       'препараты', 'дженериков', 'препарата', 'препарат', 'препарат',
       'средство', 'нейролептиков', 'препарату', 'препарату',
       'препаратов', 'препар

In [124]:
# df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :].append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)])
df.loc[(df['POS'] == 1) & (df['NER'] > 1)]
np.mean(df.loc[(df['POS'] == 1) & (df['NER'] > 1)].CASE)

2.0669642857142856

In [125]:
print(len(df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]))
df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]

19


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
446,последнему,1,1,6.0,2.0,446.0,0,3,1,1,10,7,184,0,0,0,0,2,442.0,1,0.607921,0.511480,0.980417
3539,препарат.точнее,3,3,21.0,0.0,1671.0,0,0,0,1,13,0,691,0,0,0,15,2,3.0,2,0.634102,0.863456,0.983490
5222,новейший,3,3,38.0,5.0,3354.0,0,1,1,1,10,0,1085,0,1,1,8,2,3350.0,1,0.661456,0.625717,0.981186
6857,тиапридал,5,5,23.0,6.0,659.0,0,0,1,1,0,12,958,0,0,0,0,2,659.0,1,0.700748,0.516221,0.968641
8834,это,7,7,0.0,25.0,295.0,0,0,0,1,3,12,941,0,0,0,9,3,293.0,2,0.458748,0.485954,0.988494
8928,хт,7,7,1.0,2.0,389.0,0,0,0,1,16,21,1634,1,1,1,1,2,0.0,2,0.578710,0.558414,0.988494
8987,хт,7,7,1.0,8.0,448.0,0,0,0,1,16,21,1651,1,2,3,1,2,0.0,2,0.578710,0.569288,0.988494
9086,хт,7,7,1.0,13.0,547.0,0,0,0,1,16,10,229,0,0,0,5,2,0.0,2,0.578710,0.603206,0.981244
9497,ксф,7,7,1.0,57.0,958.0,0,0,0,1,16,12,1791,0,0,0,0,2,0.0,2,0.499548,0.668588,0.982222
9935,хт,7,7,7.0,0.0,1396.0,0,0,0,1,16,21,1634,1,1,1,1,2,0.0,2,0.578710,0.558414,0.981635


In [235]:
med_df = df.loc[(df['POS'] == 1) ]\
            .append(df.loc[(df['POS'] == 5) & (df['PERSON'] == 2)]).\
            loc[:,['TOKEN_VECT', 'HEAD_VECT', 'NER']]


# med_df = df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:1409, :]\
#             .append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)]).\
#             loc[:,['TOKEN_VECT', 'HEAD_VECT', 'SENT_VECT', 'NER']]

In [236]:
med_df

,TOKEN_VECT,HEAD_VECT,NER
1,0.592381,0.549628,2
2,0.743508,0.857506,1
3,0.770621,0.578861,1
16,0.513477,0.590512,0
19,0.538471,0.461803,0
...,...,...,...
16540,0.554114,0.551750,0
16770,0.538690,0.496138,0
17357,0.538690,0.496138,2
17731,0.538690,0.579431,0


In [237]:
train, test = train_test_split(med_df, test_size=0.3, shuffle=True)
train = train.reset_index().drop(['index'], axis=1)
test = test.reset_index().drop(['index'], axis=1)
X, y, X_test, y_test = train.drop(['NER'], axis=1), \
                    train.NER, test.drop(['NER'], axis=1), test.NER

In [238]:
len(train), len(test)

(3766, 1614)

In [239]:
len(X)

3766

In [240]:
y

0       2
1       0
2       0
3       0
4       0
       ..
3761    0
3762    0
3763    0
3764    0
3765    0
Name: NER, Length: 3766, dtype: int64

In [241]:
len([i for i in y if i == 2])

167

In [242]:
y_test[:9]

0    0
1    1
2    0
3    0
4    0
5    0
6    2
7    0
8    1
Name: NER, dtype: int64

In [243]:
len([i for i in y_test if i == 2])

63

In [244]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
0  true positive
1614  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [245]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
173  true positive
55  false positive
1296  true negative
90  false negative
---------
0.7587719298245614  presicion
0.6577946768060836  recall
0.7046843177189409  f-score


In [246]:
pred 

array([0, 1, 0, ..., 0, 0, 0])

In [247]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
202  true positive
70  false positive
1280  true negative
62  false negative
---------
0.7426470588235294  presicion
0.7651515151515151  recall
0.753731343283582  f-score


In [248]:
clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
184  true positive
45  false positive
1304  true negative
81  false negative
---------
0.8034934497816594  presicion
0.6943396226415094  recall
0.7449392712550608  f-score


In [249]:
topics_sum = 0
for i in topics:
  if df.loc[df['TOPIC_NUM']==i].POST_NUM.any():
    topics_sum += df.loc[df['TOPIC_NUM']==i].POST_NUM.values[-1]
    # print(df.loc[df['TOPIC_NUM']==i].POST_NUM.values[-1])
topics_sum

240.0